<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EC%9B%90%EC%8A%A4_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [194]:
# konlpy 설치하기
!pip install konlpy

In [195]:
# konlpy를 실행하기
import konlpy

In [196]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [197]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [198]:
## 문서 ID로 실제 파일 불러오기
# 원스 (1).csv
## https://drive.google.com/file/d/1qyvxUV0Wot3WClJCJZu1E9OJ3U20byId/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1qyvxUV0Wot3WClJCJZu1E9OJ3U20byId'})
rawdata_downloaded.GetContentFile('원스 (1).csv')

In [199]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('원스 (1).csv', sep=",")

In [200]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,질럿(qeta****),이루어지는 사랑만이 아름다운 사랑이 아니라는 걸 느꼈다,10,976,67,2013-08-28 08:16:00,0
1,1,소년(kook****),Miluju tebe,9,635,55,2013-08-05 01:24:00,0
2,2,간지라기(cabi****),"흔들리는 카메라, 소음 섞인 대화, 뭔가 어설프고 어눌한 연기자들의 모습들, 이 모...",10,556,46,2014-05-14 06:39:00,0
3,3,12(iche****),영화가 끝나고 네이버 검색창에 '밀루유떼베'를 친순간 몸에 전율이 흘렀다..,10,457,34,2014-10-01 19:38:00,0
4,4,쥬피터(jupi****),지금까지 본 음악영화 중 최고가 아닌가 싶다... 왜 이제야 봤을까! 캬~,10,303,34,2013-08-20 01:31:00,0
...,...,...,...,...,...,...,...,...
6385,6385,손흥민(thsg****),이루지지 않은 사랑도 아름답지만 이루어지는 사랑이 더 아름다운 것 같아,9,2,8,2018-11-27 00:18:00,0
6386,6386,앤불린(0807****),추운 겨울 흐린날 바람이 불어 오면 밀루유떼베,10,2,8,2018-10-29 23:08:00,0
6387,6387,해지기(lww1****),따뜻하고 좋은 영화네요.,9,2,8,2018-10-28 23:08:00,0
6388,6388,광휘(hond****),한순간의 꿈과도 같은 시간 음악영화 너무 좋아..,10,1,8,2018-12-19 20:16:00,0


In [201]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [202]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,질럿(qeta****),이루어지는 사랑만이 아름다운 사랑이 아니라는 걸 느꼈다,10,976,67,2013-08-28 08:16:00,0
1,1,소년(kook****),Miluju tebe,9,635,55,2013-08-05 01:24:00,0
2,2,간지라기(cabi****),"흔들리는 카메라, 소음 섞인 대화, 뭔가 어설프고 어눌한 연기자들의 모습들, 이 모...",10,556,46,2014-05-14 06:39:00,0
3,3,12(iche****),영화가 끝나고 네이버 검색창에 '밀루유떼베'를 친순간 몸에 전율이 흘렀다..,10,457,34,2014-10-01 19:38:00,0
4,4,쥬피터(jupi****),지금까지 본 음악영화 중 최고가 아닌가 싶다... 왜 이제야 봤을까! 캬~,10,303,34,2013-08-20 01:31:00,0
...,...,...,...,...,...,...,...,...
6385,6385,손흥민(thsg****),이루지지 않은 사랑도 아름답지만 이루어지는 사랑이 더 아름다운 것 같아,9,2,8,2018-11-27 00:18:00,0
6386,6386,앤불린(0807****),추운 겨울 흐린날 바람이 불어 오면 밀루유떼베,10,2,8,2018-10-29 23:08:00,0
6387,6387,해지기(lww1****),따뜻하고 좋은 영화네요.,9,2,8,2018-10-28 23:08:00,0
6388,6388,광휘(hond****),한순간의 꿈과도 같은 시간 음악영화 너무 좋아..,10,1,8,2018-12-19 20:16:00,0


# 형태소 분석

In [203]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [204]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [205]:
# "형태소" 변수의 저장 내용 확인
형태소

[('따듯', 'Verb'),
 ('한', 'Determiner'),
 ('일상', 'Noun'),
 ('이었다', 'Verb'),
 ('.', 'Punctuation'),
 ('마지막', 'Noun'),
 ('까지', 'Josa'),
 ('눈살', 'Noun'),
 ('찌푸려지는', 'Verb'),
 ('장면', 'Noun'),
 ('하나', 'Noun'),
 ('없는', 'Adjective'),
 ('영화', 'Noun')]

In [207]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,따듯,Verb,10,1
1,한,Determiner,10,1
2,일상,Noun,10,1
3,이었다,Verb,10,1
4,.,Punctuation,10,1
5,마지막,Noun,10,1
6,까지,Josa,10,1
7,눈살,Noun,10,1
8,찌푸려지는,Verb,10,1
9,장면,Noun,10,1


In [208]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0          1  type  count
 0  이루어지는       Verb    10      1
 1     사랑       Noun    10      1
 2     만이       Josa    10      1
 3   아름다운  Adjective    10      1
 4     사랑       Noun    10      1
 5      이       Josa    10      1
 6   아니라는  Adjective    10      1
 7      걸       Noun    10      1
 8    느꼈다       Verb    10      1,         0      1  type  count
 0  Miluju  Alpha     9      1
 1    tebe  Alpha     9      1,        0            1  type  count
 0   흔들리는         Verb    10      1
 1    카메라         Noun    10      1
 2      ,  Punctuation    10      1
 3     소음         Noun    10      1
 4     섞인         Verb    10      1
 5     대화         Noun    10      1
 6      ,  Punctuation    10      1
 7     뭔가         Noun    10      1
 8   어설프고    Adjective    10      1
 9    어눌한    Adjective    10      1
 10   연기자         Noun    10      1
 11     들       Suffix    10      1
 12     의         Josa    10      1
 13    모습         Noun    10      1
 14     들       Suffix    1

In [209]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [210]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,이루어지는,Verb,10,1
1,사랑,Noun,10,1
2,만이,Josa,10,1
3,아름다운,Adjective,10,1
4,사랑,Noun,10,1
...,...,...,...,...
8,찌푸려지는,Verb,10,1
9,장면,Noun,10,1
10,하나,Noun,10,1
11,없는,Adjective,10,1


In [211]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [212]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,이루어지는,Verb,10,1
1,사랑,Noun,10,1
2,만이,Josa,10,1
3,아름다운,Adjective,10,1
4,사랑,Noun,10,1
...,...,...,...,...
8,찌푸려지는,Verb,10,1
9,장면,Noun,10,1
10,하나,Noun,10,1
11,없는,Adjective,10,1


In [213]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [214]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소    품사           분류
!      Punctuation  1      6
                    5      1
                    7      4
                    8     20
                    9     34
                          ..
힘들었다   Adjective    9      1
힘들었던   Adjective    10     1
힘들었던것  Adjective    10     1
힘들지만   Adjective    8      1
힙합     Noun         10     1
Name: 카운트, Length: 15466, dtype: int64

In [215]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [216]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소   품사          분류     
!     Punctuation 1     6
                  5     1
                  7     4
                  8    20
                  9    34
...                   ...
힘들었다  Adjective   9     1
힘들었던  Adjective   10    1
힘들었던것 Adjective   10    1
힘들지만  Adjective   8     1
힙합    Noun        10    1

[15466 rows x 1 columns]

In [217]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')